In [1]:
# 첨부파일 경로 변환 - any_approval DB 전자결재 문서용
# documents 테이블의 attaches 컬럼 처리
# end_year로 연도별 필터링

import json
import os
import shutil
from pathlib import Path
import pymysql
from sqlalchemy import create_engine

# ==================== DB 설정 ====================
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "any_approval"  # ✨ 변경

# ==================== 설정 영역 (여기를 수정하세요) ====================

# 종료연도 (필터링용)
END_YEAR = 2015  # ✨ 변경: 2025, 2024, 2023 등

# 첨부파일 원본 경로들 - 여러 경로 추가 가능
BASE_FILE_PATHS = [
    # r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\file\apr",  # ✨ brd → apr
    # r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\file\apr",  # ✨ brd → apr
    r"C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html\resource\file\apr",  # ✨ brd → apr
    # r"C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html\resource\file\apr",  # ✨ brd → apr
    # 더 추가하고 싶으면 여기에 계속 추가
]

# 첨부파일 복사본 저장 폴더 (연도별로 자동 생성)
OUTPUT_FILE_FOLDER = rf"C:\Users\LEEJUHWAN\Downloads\approval_{END_YEAR}_attachments"  # ✨ 변경

# path에서 제거할 접두사 (이 부분을 제거하고 상대경로로 변경)
REMOVE_PREFIX = r"C:\Users\LEEJUHWAN\Downloads"

# ======================================================================


def find_attachment_folder(doc_id):
    """여러 경로에서 첨부파일 폴더 찾기"""
    for base_path in BASE_FILE_PATHS:
        attachment_folder = Path(base_path) / doc_id
        if attachment_folder.exists():
            return attachment_folder
    return None


def find_file_in_folder(folder, filename):
    """폴더 내에서 파일명과 일치하는 파일 찾기 (대소문자 구분 없음)"""
    if not folder or not folder.exists():
        return None
    
    # 정확한 파일명으로 먼저 찾기
    target_file = folder / filename
    if target_file.exists() and target_file.is_file():
        return target_file
    
    # 대소문자 구분 없이 찾기
    for file in folder.iterdir():
        if file.is_file() and file.name.lower() == filename.lower():
            return file
    
    return None


def process_single_document_attachments(data, doc_index):
    """하나의 document의 첨부파일 처리"""
    
    # source_id에서 문서ID 추출
    source_id = data.get('source_id', '')  # ✨ sourceId → source_id
    if not source_id:
        print(f"⚠️ [{doc_index}] source_id 없음, 건너뜀")
        return data
    
    doc_id = source_id  # ✨ lstrip('_') 제거, 그대로 사용
    
    # attaches 배열 확인
    attaches = data.get('attaches', [])
    if not attaches:
        return data  # 첨부파일 없으면 그대로 반환
    
    # 여러 경로에서 첨부파일 폴더 찾기
    attachment_folder = find_attachment_folder(doc_id)
    
    if not attachment_folder:
        print(f"⚠️ [{doc_index}] 문서ID {doc_id} 폴더 없음 (모든 경로 검색 완료)")
        # 폴더가 없으면 모든 첨부파일의 path를 공란으로
        for attach in attaches:
            attach['path'] = ''
        return data
    
    # 복사본 저장할 문서별 폴더 생성
    output_doc_folder = Path(OUTPUT_FILE_FOLDER) / f"apr{doc_id}"  # ✨ brd → apr
    output_doc_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📄 [{doc_index}] 문서ID: {doc_id}")
    print(f"   원본: {attachment_folder}")
    print(f"   복사: {output_doc_folder}")
    print(f"   ✅ 첨부파일 {len(attaches)}개 발견")
    
    # 각 첨부파일 처리
    processed_attaches = []
    copied_count = 0
    
    for idx, attach in enumerate(attaches):
        filename = attach.get('name', '')
        if not filename:
            print(f"      ⚠️ [{idx}] 파일명 없음 - path 공란 처리")
            attach['path'] = ''
            processed_attaches.append(attach)
            continue
        
        # 원본 폴더에서 파일 찾기
        source_file = find_file_in_folder(attachment_folder, filename)
        
        if source_file:
            # 복사본 파일 경로
            dest_file = output_doc_folder / filename
            
            # 파일 복사
            try:
                shutil.copy2(source_file, dest_file)
                copied_count += 1
                print(f"      ✅ {filename}")
                
                # path를 상대경로로 변경 (REMOVE_PREFIX 제거)
                new_path = str(dest_file).replace('\\', '/')
                if REMOVE_PREFIX:
                    new_path = new_path.replace(REMOVE_PREFIX.replace('\\', '/'), '').lstrip('/')
                    new_path = new_path.replace(REMOVE_PREFIX, '').lstrip('\\')
                
                attach['path'] = new_path
                
            except Exception as e:
                print(f"      ❌ {filename} 복사 실패: {e}")
                attach['path'] = ''
        else:
            print(f"      ⚠️ 파일 없음: {filename} - path 공란 처리")
            attach['path'] = ''
        
        processed_attaches.append(attach)
    
    print(f"   📊 복사 완료: {copied_count}/{len(attaches)}개")
    
    # 수정된 attaches 적용
    data['attaches'] = processed_attaches
    return data


def process_db(end_year):
    """DB에서 특정 end_year의 데이터만 읽고 처리 후 DB에 업데이트"""
    
    # 출력 폴더 생성
    os.makedirs(OUTPUT_FILE_FOLDER, exist_ok=True)
    
    # DB 연결
    print(f"📂 DB 연결 중...")
    print(f"📂 종료연도: {end_year}")  # ✨ 변경
    print(f"📂 검색할 첨부파일 경로: {len(BASE_FILE_PATHS)}개\n")
    
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASS,
        database=DB,
        port=PORT,
        charset='utf8mb4'
    )
    
    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:
            # 1. DB에서 특정 end_year의 데이터만 읽기
            query = "SELECT * FROM documents WHERE end_year = %s"  # ✨ 변경
            cursor.execute(query, (end_year,))
            documents = cursor.fetchall()  # ✨ posts → documents
            
            total_count = len(documents)
            print(f"✅ 총 {total_count}개의 documents 발견\n")
            
            # 2. 각 document 처리
            success_count = 0
            total_attachments = 0
            
            for idx, doc in enumerate(documents, 1):  # ✨ post → doc
                try:
                    # ✨ attaches 파싱 (이미 JSON이면 그대로, 문자열이면 loads)
                    attaches_data = doc.get('attaches', [])
                    if isinstance(attaches_data, str):
                        doc['attaches'] = json.loads(attaches_data)
                    else:
                        doc['attaches'] = attaches_data if attaches_data else []
                    
                    # 첨부파일이 있는 경우만 카운트
                    if doc.get('attaches'):
                        total_attachments += len(doc.get('attaches'))
                    
                    # 기존 로직 그대로 사용
                    result = process_single_document_attachments(doc, idx)
                    
                    # attaches가 변경된 경우 DB 업데이트
                    if result.get('attaches'):
                        # attaches를 JSON 문자열로 변환
                        attaches_json = json.dumps(result['attaches'], ensure_ascii=False)
                        
                        update_query = """
                            UPDATE documents 
                            SET attaches = %s 
                            WHERE source_id = %s AND end_year = %s
                        """  # ✨ board_posts → documents, attaches_json → attaches, sourceId → source_id, type → end_year
                        cursor.execute(update_query, (attaches_json, result['source_id'], end_year))
                        success_count += 1
                    
                except Exception as e:
                    print(f"❌ [Document {idx}] 처리 오류: {e}")
                    continue
            
            # 변경사항 커밋
            connection.commit()
            
            print(f"\n{'='*60}")
            print(f"✅ 처리 완료!")
            print(f"   - 종료연도: {end_year}")  # ✨ 변경
            print(f"   - 첨부파일 있는 documents: {success_count}개")
            print(f"   - 총 첨부파일 개수: {total_attachments}개")
            print(f"   - 복사된 파일 위치: {OUTPUT_FILE_FOLDER}")
            print(f"   - DB 테이블: documents (attaches 컬럼 업데이트)")  # ✨ 변경
            print(f"{'='*60}")
    
    finally:
        connection.close()
        print("\n✅ DB 연결 종료")


# 실행
if __name__ == "__main__":
    process_db(END_YEAR)  # ✨ BOARD_TYPE → END_YEAR

📂 DB 연결 중...
📂 종료연도: 2015
📂 검색할 첨부파일 경로: 1개

✅ 총 6587개의 documents 발견


📄 [3] 문서ID: 2002913
   원본: C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html\resource\file\apr\2002913
   복사: C:\Users\LEEJUHWAN\Downloads\approval_2015_attachments\apr2002913
   ✅ 첨부파일 1개 발견
      ✅ LG전자_G-TIPIS_견적서(20110103).xls
   📊 복사 완료: 1/1개

📄 [4] 문서ID: 2002911
   원본: C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html\resource\file\apr\2002911
   복사: C:\Users\LEEJUHWAN\Downloads\approval_2015_attachments\apr2002911
   ✅ 첨부파일 2개 발견
      ✅ 견적서mis(애니파이브시스템)_1.xls
      ✅ 더존-2010[1].12.31-견적v3.pdf
   📊 복사 완료: 2/2개

📄 [9] 문서ID: 2002916
   원본: C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html\resource\file\apr\2002916
   복사: C:\Users\LEEJUHWAN\Downloads\approval_2015_attachments\apr2002916
   ✅ 첨부파일 1개 발견
      ✅ 하이패스확인증.bmp
   📊 복사 완료: 1/1개

📄 [10] 문서ID: 2002914
   원본: C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html\resource\file\apr\2002914
   복사: C:\Users\LEEJUHWAN\Downloads\approval_20